In [2]:
!pip install pyspark

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
spark = SparkSession.builder.appName('Indian food').getOrCreate()

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [18]:
df = spark.read.csv('/content/indian_food (1).csv', header=True, inferSchema=True)

In [19]:
df.show()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

**1)Find out how many unique dishes are present.**

In [20]:
df.select(col('name')).distinct().count()

255

**2) Which state has more dishes?**

In [21]:
df1=df.select(col('state')).groupBy('state').count()
df1.select('state','count').filter(col('count')==df1.agg({'count':'max'}).collect()[0][0]).show()


+-------+-----+
|  state|count|
+-------+-----+
|Gujarat|   35|
+-------+-----+



**3. How many dishes from state Karnataka?**

In [22]:
df.where(col('state')=='Karnataka').count()

6

**4. List number of unique regions**

In [24]:
df.select(col('region')).distinct().show()

+----------+
|    region|
+----------+
|        -1|
|     South|
|   Central|
|      East|
|      West|
|North East|
|     North|
|      NULL|
+----------+



**5)Count number of dishes from each region.**

In [25]:
df.select(col('region')).groupBy('region').count().show()

+----------+-----+
|    region|count|
+----------+-----+
|        -1|   13|
|      NULL|    1|
|     South|   59|
|   Central|    3|
|      East|   31|
|      West|   74|
|North East|   25|
|     North|   49|
+----------+-----+



**6)List unique 'flavor_profile' and 'course'**

In [26]:
df.select(col('flavor_profile'),col('course')).distinct().show()

+--------------+-----------+
|flavor_profile|     course|
+--------------+-----------+
|        bitter|      snack|
|         spicy|    starter|
|          sour|main course|
|            -1|      snack|
|            -1|main course|
|         sweet|main course|
|        bitter|main course|
|         spicy|      snack|
|         sweet|    dessert|
|         spicy|main course|
+--------------+-----------+



**7)Which state has more 'main course'?**

In [27]:
df1=df.select('state').groupBy('state').count()
df1.select('state','count').where(col('count')==df1.agg({'count':'max'}).collect()[0][0]).show()


+-------+-----+
|  state|count|
+-------+-----+
|Gujarat|   35|
+-------+-----+



**8. Give the %of dishes from each region.**

In [28]:
total=df.count()
df.select('region').groupBy('region').count().withColumn('percentage',col('count')/ total*100).show()


+----------+-----+-------------------+
|    region|count|         percentage|
+----------+-----+-------------------+
|        -1|   13|  5.098039215686274|
|      NULL|    1|0.39215686274509803|
|     South|   59| 23.137254901960784|
|   Central|    3| 1.1764705882352942|
|      East|   31| 12.156862745098039|
|      West|   74|  29.01960784313726|
|North East|   25|  9.803921568627452|
|     North|   49| 19.215686274509807|
+----------+-----+-------------------+



**9)List the states which has more dishes from each region.**

In [29]:
df1=df.select('state','region').groupBy('state','region').count()
df1.select('state','region','count').filter(col('count')==df1.agg({'count':'max'}).collect()[0][0]).show()


+-------+------+-----+
|  state|region|count|
+-------+------+-----+
|Gujarat|  West|   35|
+-------+------+-----+



WEATHER PROGRAM

In [52]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Weather Analysis').getOrCreate()

In [53]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [54]:
df = spark.read.csv('/content/weather_data.csv', header=True, inferSchema=True)

In [55]:
df = df.select([col for col in df.columns if col != '_c0'])
df.show()

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|
|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|
|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|
|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|
|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|
|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|
|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|
|1908|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|
|1909|17.79|19.05| 23.4|25.76|27.97|27.67|26.56|26.43|25.47|24.37|22.01|18.86|
|1910|18.14|19.72| 22.9|25.96|28.36|27.72|26.93|26.6

**1. Add column average temperature for each year**

In [56]:
df2 = df.withColumn('Average',(col('JAN')+col('FEB')+col('MAR')+col('APR')+col('MAY')+col('JUN')+col('JUL')+col('AUG')+col('SEP')+col('OCT')+col('NOV')+col('DEC'))/12)
df2.show()

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+
|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|           Average|
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+
|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|24.224166666666665|
|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|24.326666666666664|
|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|23.980833333333333|
|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|23.849999999999998|
|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|           23.9325|
|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|24.099999999999998|
|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|24.007499999999997|
|1908|18.35|19.73|22

**2. Add row average temperature of each month**

In [57]:
df2 = df.select([cols for cols in df.columns if cols != 'YEAR'])
avg_dic = {}
for i in df2.columns:
  avg_dic[i] = df.select(i).agg({i:'avg'}).collect()[0][0]
avg_dic

{'JAN': 18.42324786324786,
 'FEB': 20.137948717948717,
 'MAR': 23.434188034188036,
 'APR': 26.514102564102565,
 'MAY': 28.386410256410255,
 'JUN': 28.300940170940162,
 'JUL': 27.369230769230775,
 'AUG': 26.940085470085464,
 'SEP': 26.342649572649577,
 'OCT': 24.742051282051282,
 'NOV': 21.765726495726508,
 'DEC': 19.173333333333332}

In [58]:
new_row = Row(_c0='Average', YEAR='----', JAN=avg_dic['JAN'], FEB=avg_dic['FEB'], MAR=avg_dic['MAR'], APR=avg_dic['APR'],
                                          MAY=avg_dic['MAY'], JUN=avg_dic['JUN'], JUL=avg_dic['JUL'], AUG=avg_dic['AUG'],
                                          SEP=avg_dic['SEP'], OCT=avg_dic['OCT'], NOV=avg_dic['NOV'], DEC=avg_dic['DEC']
             )
new_row_df = spark.createDataFrame([new_row])
df_new = spark.read.csv('/content/weather_data.csv',header=True,inferSchema=True)
df_new = df_new.union(new_row_df)
spark.createDataFrame(df_new.tail(10)).show()

+-------+----+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|    _c0|YEAR|              JAN|               FEB|               MAR|               APR|               MAY|               JUN|               JUL|               AUG|               SEP|               OCT|               NOV|               DEC|
+-------+----+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|    108|2009|            19.79|             21.66|             24.55|             27.35|             28.71|             28.77|             27.83|             27.85|             27.11|              25.2|             22.31|             20.22|
|    109|2010|            19.15|

**3. Add columns Min and Max for each year with corresponding values**

In [59]:
df3 = df.withColumn('Min',least(col('JAN'),col('FEB'),col('MAR'),col('APR'),col('MAY'),col('JUN'),col('JUL'),col('AUG'),col('SEP'),col('OCT'),col('NOV'),col('DEC')))\
         .withColumn('Max',greatest(col('JAN'),col('FEB'),col('MAR'),col('APR'),col('MAY'),col('JUN'),col('JUL'),col('AUG'),col('SEP'),col('OCT'),col('NOV'),col('DEC')))
df3.show()

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|  Min|  Max|
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|17.99| 28.6|
|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|18.78|28.68|
|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|18.29|28.41|
|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|17.77|27.85|
|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71| 17.4|28.69|
|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55| 17.5|29.06|
|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|18.46|27.66|
|1908|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|18.15|28.49|

**4. Create dataframe called 'decade', storing average temperatures of each month in the respective decade. Decade 1910 is from 1901 to 1910.**

In [60]:
decade = df.groupBy(floor(col("YEAR") / 10) * 10).agg(avg("JAN").alias('JAN'),avg("FEB").alias('FEB'),avg("MAR").alias('MAR'),avg("APR").alias('APR'),avg("MAY").alias('MAY'),
    avg("JUN").alias('JUN'),avg("JUL").alias('JUL'),avg("AUG").alias('AUG'),avg("SEP").alias('SEP'),avg("OCT").alias('OCT'),avg("NOV").alias('NOV'),avg("DEC").alias('DEC'))

decade.withColumnRenamed('(floor((YEAR / 10)) * 10)','YEAR').orderBy('YEAR').show()


+----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|YEAR|               JAN|               FEB|               MAR|               APR|               MAY|               JUN|               JUL|               AUG|               SEP|               OCT|               NOV|               DEC|
+----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|1900|18.154444444444444| 19.34777777777778| 22.81666666666667|             26.32|28.184444444444445|28.203333333333337|             27.31|26.766666666666666| 26.04222222222222|24.736666666666668|21.585555555555555| 18.73222222222222|
|1910|            18.306|            19.673|            22.7

**5. Which year is the hottest year? (Based on avg temp)**

In [61]:
df2.show()

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|
| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|
|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|
|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|
| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|
| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|
|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|
|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|
|17.79|19.05| 23.4|25.76|27.97|27.67|26.56|26.43|25.47|24.37|22.01|18.86|
|18.14|19.72| 22.9|25.96|28.36|27.72|26.93|26.61|25.98|24.04|20.72|18.05|
|18.52|19.18|22.05| 26.0|28.55|28.02|2

**6. Which year is the coldest year? (Based on avg temp)**

In [67]:
df2.select('YEAR#1900').withColumnRenamed('YEAR#1900', 'YEAR').where(col('Average')==df2.select('Average').agg({'Average':'min'}).collect()[0][0]).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `YEAR#1900` cannot be resolved. Did you mean one of the following? [`MAR`, `APR`, `AUG`, `DEC`, `FEB`].;
'Project ['YEAR#1900]
+- Project [JAN#1901, FEB#1902, MAR#1903, APR#1904, MAY#1905, JUN#1906, JUL#1907, AUG#1908, SEP#1909, OCT#1910, NOV#1911, DEC#1912]
   +- Project [YEAR#1900, JAN#1901, FEB#1902, MAR#1903, APR#1904, MAY#1905, JUN#1906, JUL#1907, AUG#1908, SEP#1909, OCT#1910, NOV#1911, DEC#1912]
      +- Relation [_c0#1899,YEAR#1900,JAN#1901,FEB#1902,MAR#1903,APR#1904,MAY#1905,JUN#1906,JUL#1907,AUG#1908,SEP#1909,OCT#1910,NOV#1911,DEC#1912] csv


**7. Find out which years recorded minimum and maximum temperatures. (Not based on avg temp)**

In [63]:
df3.select('YEAR').where(col('Min')==df3.select('Min').agg({'Min':'min'}).collect()[0][0]).show()
df3.select('YEAR').where(col('Max')==df3.select('Max').agg({'Max':'max'}).collect()[0][0]).show()

+----+
|YEAR|
+----+
|1918|
+----+

+----+
|YEAR|
+----+
|1921|
+----+



**8. How much raise is observed in min and max temperature for each month?**

In [64]:
df8 = df.select([cols for cols in df.columns if cols != 'YEAR'])
for i in df8.columns:
  print(f"{i}:{df8.select(i).agg({i:'max'}).collect()[0][0]-df.select(i).agg({i:'min'}).collect()[0][0]}")

JAN:3.6700000000000017
FEB:5.789999999999999
MAR:4.829999999999998
APR:4.719999999999999
MAY:3.8100000000000023
JUN:2.5500000000000007
JUL:1.9899999999999984
AUG:1.9600000000000009
SEP:2.6400000000000006
OCT:3.719999999999999
NOV:3.330000000000002
DEC:3.91
